<a href="https://colab.research.google.com/github/BenguBarisBalkan/AIN-311_Project/blob/main/web_scaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install bs4
!pip install lxml

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
url = "https://www.spritmonitor.de/en/evaluation/co2_efficient_cars.html"

In [5]:
response = requests.get(url)

In [6]:
if response.status_code == 200:
  html = response.text
  soup = BeautifulSoup(html, "lxml")

else:
  print("Failed")

In [7]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN" "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="co2 efficiency, most co2 efficient cars, co2 emissions, reduce co2 emissions" name="keywords"/>
  <meta content="Top 50 most CO2 efficient cars" name="description"/>
  <meta content="https://www.spritmonitor.de/pics/logo/spritmonitor_thumbnail.png" property="og:image"/>
  <title>
   The most CO2 efficient cars - Spritmonitor.de
  </title>
  <base href="https://www.spritmonitor.de/"/>
  <link href="/sc/css/v2088/evaluation.css" media="screen" rel="stylesheet" type="text/css"/>
  <link href="/sc/css/v2088/main.css" media="all" rel="stylesheet" type="text/css"/>
  <link href="/sc/css/v2088/screen.css" media="screen" rel="stylesheet" type="text/css"/>
  <link href="/sc/css/v2088/print.css" media="print" rel="stylesheet" type="text/css"/>
  <link href="https://www.spritmonitor.de/en/eval

In [15]:
tbody = soup.find('tbody')
print(tbody)

<tbody>
<tr>
<td class="number">1</td>
<td><a href="/en/overview/113-Dacia/1958-Spring.html?fueltype=5">Dacia Spring</a> (85)</td>
<td>Electricity</td>
<td class="co2">67</td>
</tr>
<tr>
<td class="number">2</td>
<td><a href="/en/overview/50-Volkswagen/1234-Up!.html?fueltype=5">Volkswagen Up!</a> (225)</td>
<td>Electricity</td>
<td class="co2">68</td>
</tr>
<tr>
<td class="number">3</td>
<td><a href="/en/overview/19-Hyundai/1554-IONIQ.html?fueltype=5">Hyundai IONIQ</a> (205)</td>
<td>Electricity</td>
<td class="co2">69</td>
</tr>
<tr>
<td class="number">4</td>
<td><a href="/en/overview/50-Volkswagen/452-Golf.html?fueltype=5">Volkswagen Golf</a> (198)</td>
<td>Electricity</td>
<td class="co2">70</td>
</tr>
<tr>
<td class="number">5</td>
<td><a href="/en/overview/41-Renault/374-Twingo.html?fueltype=5">Renault Twingo</a> (68)</td>
<td>Electricity</td>
<td class="co2">71</td>
</tr>
<tr>
<td class="number">6</td>
<td><a href="/en/overview/45-Skoda/1289-Citigo.html?fueltype=5">Skoda Citigo</

In [18]:
for x in tbody:
  print(x)



<tr>
<td class="number">1</td>
<td><a href="/en/overview/113-Dacia/1958-Spring.html?fueltype=5">Dacia Spring</a> (85)</td>
<td>Electricity</td>
<td class="co2">67</td>
</tr>


<tr>
<td class="number">2</td>
<td><a href="/en/overview/50-Volkswagen/1234-Up!.html?fueltype=5">Volkswagen Up!</a> (225)</td>
<td>Electricity</td>
<td class="co2">68</td>
</tr>


<tr>
<td class="number">3</td>
<td><a href="/en/overview/19-Hyundai/1554-IONIQ.html?fueltype=5">Hyundai IONIQ</a> (205)</td>
<td>Electricity</td>
<td class="co2">69</td>
</tr>


<tr>
<td class="number">4</td>
<td><a href="/en/overview/50-Volkswagen/452-Golf.html?fueltype=5">Volkswagen Golf</a> (198)</td>
<td>Electricity</td>
<td class="co2">70</td>
</tr>


<tr>
<td class="number">5</td>
<td><a href="/en/overview/41-Renault/374-Twingo.html?fueltype=5">Renault Twingo</a> (68)</td>
<td>Electricity</td>
<td class="co2">71</td>
</tr>


<tr>
<td class="number">6</td>
<td><a href="/en/overview/45-Skoda/1289-Citigo.html?fueltype=5">Skoda Citi

In [31]:
trs = tbody.find_all('tr')

for tr in trs:
  info = tr.find_all('td')
  print(f"{info[0].text: <5} {info[1].text: <30} {info[2].text: <20} {info[3].text: <20}")

1     Dacia Spring (85)              Electricity          67                  
2     Volkswagen Up! (225)           Electricity          68                  
3     Hyundai IONIQ (205)            Electricity          69                  
4     Volkswagen Golf (198)          Electricity          70                  
5     Renault Twingo (68)            Electricity          71                  
6     Skoda Citigo (58)              Electricity          72                  
7     Smart Fortwo (144)             Electricity          73                  
8     Hyundai Kona (225)             Electricity          76                  
9     Nissan Leaf (169)              Electricity          78                  
10    BMW i3 (253)                   Electricity          78                  
11    Fiat 500 (142)                 Electricity          79                  
12    Renault ZOE (478)              Electricity          80                  
13    Kia Soul (59)                  Electricity    

In [41]:
links = []
for tr in trs:
  info = tr.find_all('td')
  link = "https://www.spritmonitor.de/" + info[1].find('a').get('href').replace(' ', '_')
  links.append(link)
  print(f"{link}")

https://www.spritmonitor.de//en/overview/113-Dacia/1958-Spring.html?fueltype=5
https://www.spritmonitor.de//en/overview/50-Volkswagen/1234-Up!.html?fueltype=5
https://www.spritmonitor.de//en/overview/19-Hyundai/1554-IONIQ.html?fueltype=5
https://www.spritmonitor.de//en/overview/50-Volkswagen/452-Golf.html?fueltype=5
https://www.spritmonitor.de//en/overview/41-Renault/374-Twingo.html?fueltype=5
https://www.spritmonitor.de//en/overview/45-Skoda/1289-Citigo.html?fueltype=5
https://www.spritmonitor.de//en/overview/32-Smart/296-Fortwo.html?fueltype=5
https://www.spritmonitor.de//en/overview/19-Hyundai/1638-Kona.html?fueltype=5
https://www.spritmonitor.de//en/overview/33-Nissan/1296-Leaf.html?fueltype=5
https://www.spritmonitor.de//en/overview/6-BMW/1312-i3.html?fueltype=5
https://www.spritmonitor.de//en/overview/16-Fiat/761-500.html?fueltype=5
https://www.spritmonitor.de//en/overview/41-Renault/1302-ZOE.html?fueltype=5
https://www.spritmonitor.de//en/overview/23-Kia/1147-Soul.html?fueltype=

In [42]:
with open('spritmonitor_links.txt', 'w') as file:
  for link in links:
    file.write(link + '\n')